In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("cleaned_telco_churn.csv")

In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,No phone service,DSL,0,...,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,No,DSL,1,...,1,0,0,0,One year,0,Mailed check,56.95,1889.50,0
2,3668-QPYBK,1,0,0,0,2,1,No,DSL,1,...,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,No phone service,DSL,1,...,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,No,Fiber optic,0,...,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1


In [5]:
df.drop("customerID",axis=1)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,No phone service,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,1,0,0,0,34,1,No,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.50,0
2,1,0,0,0,2,1,No,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,1,0,0,0,45,0,No phone service,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,0,0,0,0,2,1,No,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,24,1,Yes,DSL,1,0,1,1,1,1,One year,1,Mailed check,84.80,1990.50,0
7039,0,0,1,1,72,1,Yes,Fiber optic,0,1,1,0,1,1,One year,1,Credit card (automatic),103.20,7362.90,0
7040,0,0,1,1,11,0,No phone service,DSL,1,0,0,0,0,0,Month-to-month,1,Electronic check,29.60,346.45,0
7041,1,1,1,0,4,1,Yes,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Mailed check,74.40,306.60,1


In [6]:
df["tenure_group"] = pd.cut(
    df["tenure"],
    bins=[0, 12, 24, 36, 48, 60, 72],
    labels=["0-12", "13-24", "25-36", "37-48", "49-60", "61-72"]
)

Binning Customer Tenure
This code performs discretization (binning) on the tenure column to transform continuous numerical values into categorical groups.

Method: Uses pd.cut() to segment the customer duration into specific yearly ranges.

Bins: Data is divided into 12-month intervals (e.g., Year 1, Year 2, up to 72 months).

Categorization:

0–12: 1st Year

13–24: 2nd Year

...and so on.

Purpose: This helps in identifying patterns based on "customer lifecycles" rather than individual months, making it easier to visualize churn trends over time.

In [7]:
df["new_customer"] = (df["tenure"]<12).astype(int)

## What does “New Customer Flag” mean?

A new customer flag is a binary feature that answers one simple business question:

Is this customer new or not?

In churn problems, new customers behave very differently from long-term customers.

In your Telco data:

Customers with low tenure churn much more

EDA already showed this

So we create a feature that explicitly tells the model:

1 → new customer (high churn risk)

0 → existing customer

In [8]:
df["new_customer"].head()

0    1
1    0
2    1
3    0
4    1
Name: new_customer, dtype: int32

In [9]:
mid=df["MonthlyCharges"].median()

In [10]:
df["High_pay_customer"] = (df["MonthlyCharges"]>mid).astype(int)

Feature Engineering: Customer Segmentation by Spend
This code block creates a new binary classification feature based on the Monthly Charges of customers:

Logic: It calculates the median value of the MonthlyCharges column.

Transformation: It compares each customer's monthly charge against this median.

Result: A new column, High_pay_customer, is added where:

1 represents customers who pay above the median.

0 represents customers who pay at or below the median.

In [11]:
df["High_pay_customer"]

0       0
1       0
2       0
3       0
4       1
       ..
7038    1
7039    1
7040    0
7041    1
7042    1
Name: High_pay_customer, Length: 7043, dtype: int32

In [12]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'tenure_group', 'new_customer', 'High_pay_customer'],
      dtype='object')

In [13]:
df[["tenure","TotalCharges"]].head()

,tenure,TotalCharges
0,1,29.85
1,34,1889.50
2,2,108.15
3,45,1840.75
4,2,151.65


In [14]:
df["charges_per_tenure"] = df["TotalCharges"] / df["tenure"].replace(0, 1)
df["charges_per_tenure"].head()

0    29.850000
1    55.573529
2    54.075000
3    40.905556
4    75.825000
Name: charges_per_tenure, dtype: float64

Feature Preview: Charges per Tenure
This code displays the first few rows of the charges_per_tenure feature. This derived metric is typically used to represent the average cost intensity over the duration of a customer's contract.

Purpose: To verify the calculation of the ratio between total/monthly charges and the customer's tenure.

Insight: High values in this column indicate customers who contribute significant revenue in a relatively short period.

In [15]:
service_col = [
               'OnlineSecurity', 
               'OnlineBackup',
               'DeviceProtection',
               'TechSupport',
               'StreamingTV',
               'StreamingMovies']
df["Customer_Engagement"] = df[service_col].sum(axis=1)

Feature Engineering: Customer Engagement Score
This code calculates an Engagement Score by aggregating the additional services subscribed to by each customer.

Service Selection: Defines a list of service-related columns, including security, backup, protection, support, and streaming options.

Logic: Uses .sum(axis=1) to count the total number of services active for each row (customer).

Outcome: Creates a new numerical feature, Customer_Engagement, which serves as a proxy for customer loyalty—the higher the score, the more integrated the customer is into the company's ecosystem.

In [16]:
df["Customer_Engagement"]

0       1
1       2
2       2
3       3
4       0
       ..
7038    5
7039    4
7040    1
7041    0
7042    5
Name: Customer_Engagement, Length: 7043, dtype: int64

In [17]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'tenure_group', 'new_customer', 'High_pay_customer',
       'charges_per_tenure', 'Customer_Engagement'],
      dtype='object')

In [18]:
df["Contract"].head(2)

0    Month-to-month
1          One year
Name: Contract, dtype: object

In [19]:
df["is_electronic_check"] = (df["PaymentMethod"] == "Electronic check").astype(int)


In [20]:
df["is_electronic_check"]

0       1
1       0
2       0
3       0
4       1
       ..
7038    0
7039    0
7040    1
7041    0
7042    0
Name: is_electronic_check, Length: 7043, dtype: int32

Feature Engineering: Payment Method Indicator
This code creates a binary indicator specifically for the "Electronic check" payment method.

Target: Identifies rows where the PaymentMethod is exactly "Electronic check".

Logic: Converts the boolean result (True/False) into an integer (1 for electronic check, 0 for all other methods).

Rationale: In many churn datasets, "Electronic check" is often a high-risk category for customer turnover; isolating it allows the model to weight this specific behavior more effectively.

In [21]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group,new_customer,High_pay_customer,charges_per_tenure,Customer_Engagement,is_electronic_check
0,7590-VHVEG,0,0,1,0,1,0,No phone service,DSL,0,...,Electronic check,29.85,29.85,0,0-12,1,0,29.850000,1,1
1,5575-GNVDE,1,0,0,0,34,1,No,DSL,1,...,Mailed check,56.95,1889.50,0,25-36,0,0,55.573529,2,0
2,3668-QPYBK,1,0,0,0,2,1,No,DSL,1,...,Mailed check,53.85,108.15,1,0-12,1,0,54.075000,2,0
3,7795-CFOCW,1,0,0,0,45,0,No phone service,DSL,1,...,Bank transfer (automatic),42.30,1840.75,0,37-48,0,0,40.905556,3,0
4,9237-HQITU,0,0,0,0,2,1,No,Fiber optic,0,...,Electronic check,70.70,151.65,1,0-12,1,1,75.825000,0,1


In [23]:
df.to_csv("Feature_Engineering_churn.csv",index=False)